# [1] Load Experiment Configs

Change the configs in `run_config.json`.

In [ ]:
from load_configs import get_all_exp_args, load_run_config

# load the TimeFuse exp configs
run_configs = load_run_config("run_config.json", verbose=True)

# load the base model exp args (used for TSLib models) for base model train and inference
all_exp_args = get_all_exp_args(
    datasets=run_configs["datasets"],
    models=run_configs["models"],
    forecast_settings=run_configs["forecast_settings"],
    override_args=run_configs["override_args"],
    verbose=False,
)

all_exp_args

==========Run config file: run_config.json==========
[2] datasets              : ['ETTh1', 'ETTh2']
[6] models                : ['DLinear', 'PatchTST', 'TimesNet', 'PAttn', 'TimeMixer', 'TimeXer']
[1] forecast_settings     : [[96, 48, 96]]
override_args             : {'use_gpu': True, 'gpu': 0}


{'ETTh1_DLinear_96_48_96': Namespace(task_name='long_term_forecast', is_training=1, model_id='ETTh1_96_96', model='DLinear', data='ETTh1', root_path='./dataset/long_term_forecast/', data_path='ETTh1.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', seq_len=96, label_len=48, pred_len=96, seasonal_patterns='Monthly', inverse=False, mask_rate=0.25, anomaly_ratio=0.25, expand=2, d_conv=4, top_k=5, num_kernels=6, enc_in=7, dec_in=7, c_out=7, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=3, distil=True, dropout=0.1, embed='timeF', activation='gelu', channel_independence=1, decomp_method='moving_avg', use_norm=1, down_sampling_layers=3, down_sampling_window=2, down_sampling_method='avg', seg_len=48, num_workers=64, itr=1, train_epochs=10, batch_size=32, patience=3, learning_rate=0.0001, des="'Exp'", loss='MSE', lradj='type1', use_amp=False, use_gpu=True, gpu=0, gpu_type='cuda', use_multi_gpu=False, devices='0,1,2,3', p_hidden_dims=[128

# [2] Base Model Training

Train base models for each dataset and forecast settin, the trained model will be saved in the `checkpoints` folder.
- If the checkpoint folder does not exist, it will be created automatically.
- If the checkpoint folder already exists, the training will be skipped.

In [2]:
from exp.exp_fuse_forecasting import Exp_Fuse_Forecasting

for exp_name, args in all_exp_args.items():
    setting = "{}_{}_{}/{}_{}_dmodel{}_epoch{}".format(
        args.seq_len,
        args.label_len,
        args.pred_len,
        args.data_name,
        args.model,
        args.d_model,
        args.train_epochs,
    )
    exp = Exp_Fuse_Forecasting(args)
    print(f"[Device: {exp.device}] Base model training {setting}")
    model, vali_loss, test_loss = exp.train(
        setting=setting,
        verbose=False,
        tqdm_disable=False,
        save_model=True,
        override_saved_model=False,
        raise_fwd_error=True,
    )

[Device: cuda:0] Base model training 96_48_96/ETTh1_DLinear_dmodel512_epoch10
[Base Model Train] Model already trained, loading from ./checkpoints/96_48_96/ETTh1_DLinear_dmodel512_epoch10 | Set override_saved_model=True to train and override.
[Device: cuda:0] Base model training 96_48_96/ETTh1_PatchTST_dmodel512_epoch10
[Base Model Train] Model already trained, loading from ./checkpoints/96_48_96/ETTh1_PatchTST_dmodel512_epoch10 | Set override_saved_model=True to train and override.
[Device: cuda:0] Base model training 96_48_96/ETTh1_TimesNet_dmodel16_epoch10
[Base Model Train] Model already trained, loading from ./checkpoints/96_48_96/ETTh1_TimesNet_dmodel16_epoch10 | Set override_saved_model=True to train and override.
[Device: cuda:0] Base model training 96_48_96/ETTh1_PAttn_dmodel512_epoch10
[Base Model Train] Model already trained, loading from ./checkpoints/96_48_96/ETTh1_PAttn_dmodel512_epoch10 | Set override_saved_model=True to train and override.
[Device: cuda:0] Base model tr

# [3] Meta-training Data Extraction

Extract meta-training data based on the trained base models. The extracted data will be saved in the `meta_data/` folder.

Given a training time series pair (`X_in`, `X_out`) and k base models, we extract the following meta-training data:
- `x_meta`: the meta-features of the input time series `X_in`
- `y_model_preds`: the predictions of the k base models (i.e., their preditions of `X_out`)
- `y_true`: the ground truth `X_out`

In [ ]:
from exp.exp_fuse_forecasting import Exp_Fuse_Forecasting
from utils.save_array import save_arr
from utils.metrics import metric
import os
import time
import numpy as np
import pandas as pd
import torch


def extract_input_meta_feats(
    run_configs,
    all_exp_args,
    meta_train_root,
    dataset_name,
    split_name,
    seq_len,
    force_override=False,
):
    # Extract input temporal meta feature
    meta_file_path = (
        f"{meta_train_root}/{dataset_name}_{split_name}/x_meta_{seq_len}.h5"
    )
    if not force_override and os.path.exists(meta_file_path):
        print(
            f"[Input Meta Feat Extract] File {meta_file_path} already exists, skipping..."
        )
        return
    else:
        print(
            f"[Input Meta Feat Extract] Extracting {dataset_name}-{split_name} meta features..."
        )

    args = all_exp_args[
        f"{dataset_name}_{run_configs['models'][0]}_{seq_len}_{label_len}_{pred_len}"
    ]  # for initializing the exp class only
    exp = Exp_Fuse_Forecasting(args)
    df_x_meta = exp.get_test_meta_feature(
        split_name=split_name
    )  # extract input time series meta features
    save_arr(
        arr=df_x_meta.values,
        file_path=meta_file_path,
        file_type="h5",
        create_dir=True,
    )
    return


def extract_output_pred_true(
    run_configs,
    dataset_name,
    meta_train_root,
    split_name,
    seq_len,
    label_len,
    pred_len,
    force_override=False,
):
    # Extract and save model predictions & ground truth
    postfix = f"{seq_len}_{label_len}_{pred_len}"
    pred_file_path = (
        f"{meta_train_root}/{dataset_name}_{split_name}/y_pred_{postfix}.h5"
    )
    true_file_path = (
        f"{meta_train_root}/{dataset_name}_{split_name}/y_true_{postfix}.h5"
    )
    if (
        not force_override
        and os.path.exists(pred_file_path)
        and os.path.exists(true_file_path)
    ):
        print(
            f"[Output Pred & True Extract] Files {pred_file_path} and {true_file_path} already exist, skipping..."
        )
        return
    else:
        print(
            f"[Output Pred & True Extract] Extracting {dataset_name}-{split_name} predictions and ground truth..."
        )

    data_preds = {}
    for model_name in run_configs["models"]:

        args = all_exp_args[
            f"{dataset_name}_{model_name}_{seq_len}_{label_len}_{pred_len}"
        ]
        exp = Exp_Fuse_Forecasting(args)

        setting = "{}_{}_{}/{}_{}_dmodel{}_epoch{}".format(
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.data_name,
            args.model,
            args.d_model,
            args.train_epochs,
        )

        start_time = time.time()
        print(
            f"[Output Pred & True Extract] Inferencing ({exp.device}): ",
            setting,
            end=" ... \t",
        )
        (
            preds,
            trues,
            mae,
            mse,
            rmse,
            mape,
            mspe,
        ) = exp.test(  # predict with saved model
            setting=setting,
            split_name=split_name,
            load_saved_model=True,
            verbose=False,
        )
        print(f"Done in {time.time() - start_time:.2f} seconds")
        data_preds[model_name] = preds

        del exp
        torch.cuda.empty_cache()

    # rearrange preds dimension
    print(f"[Output Pred & True Extract] Rearranging preds dimension ...", end="")
    start_time = time.time()
    all_model_preds = np.array(
        [data_preds[model_name] for model_name in run_configs["models"]]
    ).transpose(1, 0, 2, 3)
    print(f"Done in {time.time() - start_time:.2f} seconds")

    postfix = f"{seq_len}_{label_len}_{pred_len}"
    pred_file_path = (
        f"{meta_train_root}/{dataset_name}_{split_name}/y_pred_{postfix}.h5"
    )
    true_file_path = (
        f"{meta_train_root}/{dataset_name}_{split_name}/y_true_{postfix}.h5"
    )
    save_arr(
        arr=all_model_preds,
        file_path=pred_file_path,
        file_type="h5",
        create_dir=True,
    )
    save_arr(
        arr=trues,
        file_path=true_file_path,
        file_type="h5",
        create_dir=True,
    )
    return


split_names = ["val", "test"]
meta_train_root = "./meta_data"

for dataset_name in run_configs["datasets"]:
    for seq_len, label_len, pred_len in run_configs["forecast_settings"]:
        for split_name in split_names:
            # Extract input temporal meta feature
            extract_input_meta_feats(
                run_configs=run_configs,
                all_exp_args=all_exp_args,
                meta_train_root=meta_train_root,
                dataset_name=dataset_name,
                split_name=split_name,
                seq_len=seq_len,
                force_override=False,
            )

            # Extract and save model predictions & ground truth
            extract_output_pred_true(
                run_configs=run_configs,
                dataset_name=dataset_name,
                meta_train_root=meta_train_root,
                split_name=split_name,
                seq_len=seq_len,
                label_len=label_len,
                pred_len=pred_len,
                force_override=False,
            )

[Input Meta Feat Extract] File ./meta_data/ETTh1_val/x_meta_96.h5 already exists, skipping...
[Output Pred & True Extract] Files ./meta_data/ETTh1_val/y_pred_96_48_96.h5 and ./meta_data/ETTh1_val/y_true_96_48_96.h5 already exist, skipping...
[Input Meta Feat Extract] File ./meta_data/ETTh1_test/x_meta_96.h5 already exists, skipping...
[Output Pred & True Extract] Files ./meta_data/ETTh1_test/y_pred_96_48_96.h5 and ./meta_data/ETTh1_test/y_true_96_48_96.h5 already exist, skipping...
[Input Meta Feat Extract] File ./meta_data/ETTh2_val/x_meta_96.h5 already exists, skipping...
[Output Pred & True Extract] Files ./meta_data/ETTh2_val/y_pred_96_48_96.h5 and ./meta_data/ETTh2_val/y_true_96_48_96.h5 already exist, skipping...
[Input Meta Feat Extract] File ./meta_data/ETTh2_test/x_meta_96.h5 already exists, skipping...
[Output Pred & True Extract] Files ./meta_data/ETTh2_test/y_pred_96_48_96.h5 and ./meta_data/ETTh2_test/y_true_96_48_96.h5 already exist, skipping...


,dataset_name,seq_len_label_len_pred_len,split_name,model_scores


# [4] TimeFuse: Fusor training and evaluation

Train the TimeFuse fusor model on the meta-training data.

The fusor takes input meta-features `x_meta` and outputs a weight vector `w` with length k for the k base models. The weight vector is used to combine the predictions of the k base models to make the final prediction.

In [4]:
from timefuse import (
    ModelFusor,
    get_datasets_and_loaders,
    get_scaler,
    test_fusor,
    print_test_scores,
    get_length_aligned_loaders,
)
import time
import random
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import time
import tqdm

random_seed = 2021
n_epochs = 5  # meta training epochs
batch_size = 64  # meta batch size
learning_rate = 0.0005  # fusor learning rate
num_workers = 1
gpu_id = 3  # the gpu id to use for meta training
device = torch.device(f"cuda:{gpu_id}" if torch.cuda.is_available() else "cpu")

meta_train_data_names = [
    f"{dataname}_val" for dataname in run_configs["datasets"]
]  # for meta training
meta_test_data_names = [
    f"{dataname}_test" for dataname in run_configs["datasets"]
]  # for meta testing

dim_meta_feats = 22  # fusor input dim
dim_model_weights = len(
    run_configs["models"]
)  # fusor output dim, i.e., number of models

meta_scaler = get_scaler("standard")  # input meta feature scaler

# Initialize model and optimizer
fusor = ModelFusor(input_dim=dim_meta_feats, output_dim=dim_model_weights)
fusor.to(device)
optimizer = optim.Adam(fusor.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
criterion = nn.SmoothL1Loss(beta=0.01)

print(
    f" TIMEFUSE Meta Training Config ".center(50, "=") + "\n"
    f"loss={criterion}, dim_meta_feats={dim_meta_feats}, dim_model_weights={dim_model_weights}\n"
    f"n_epochs={n_epochs}, batch_size={batch_size}, learning_rate={learning_rate}, device={device}\n"
)

========= TIMEFUSE Meta Training Config ==========
loss=SmoothL1Loss(), dim_meta_feats=22, dim_model_weights=6
n_epochs=5, batch_size=64, learning_rate=0.0005, device=cuda:3



In [14]:
from utils.metrics import metric, ALL_METRICS


for forecast_settings in run_configs["forecast_settings"]:

    random.seed(random_seed)
    torch.manual_seed(random_seed)
    np.random.seed(random_seed)

    training_step = forecast_settings[2]

    print(f"\n////// Forecast: {forecast_settings} //////\n")

    # Initialize data loaders and datasets
    dataload_kwargs = {
        "forecast_setting": forecast_settings,
        "subset_seed": random_seed,
        "num_workers": num_workers,
    }
    meta_train_datasets, meta_train_loaders = get_datasets_and_loaders(
        meta_train_data_names,
        batch_size=batch_size,
        shuffle=True,
        **dataload_kwargs,
    )
    meta_test_datasets, meta_test_loaders = get_datasets_and_loaders(
        meta_test_data_names,
        batch_size=512,
        shuffle=False,
        **dataload_kwargs,
    )
    # Get aligned length loaders
    aligned_train_loaders = get_length_aligned_loaders(
        meta_train_datasets,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
    )

    # compute the best single performance for each dataset
    model_scores = {}
    for data_name, meta_dataset in meta_test_datasets.items():
        model_preds = meta_dataset.y_model_preds
        true = meta_dataset.y_true
        model_scores[data_name] = {}
        for model_id, model_name in enumerate(run_configs["models"]):
            model_score = metric(
                pred=model_preds[:, model_id, :, :],
                true=true,
                return_dict=True,
            )
            model_scores[data_name][model_name] = model_score
    test_best_single_perf = {}
    for data_name, data_scores in model_scores.items():
        test_best_single_perf[data_name] = {}
        for metric_name in ALL_METRICS:
            scores = [v[metric_name] for k, v in data_scores.items()]
            best_score = min(scores)
            test_best_single_perf[data_name][metric_name] = best_score

    # Get the scaler for the meta-features
    start_time = time.time()
    print("Fitting the scaler for the meta-features ... ", end="")
    scaler = get_scaler("standard")
    all_meta_x = np.concatenate(
        [dataset.x_meta for dataset in meta_train_datasets.values()]
    )
    scaler.fit(all_meta_x)
    print(f"done in {time.time() - start_time:.2f}s")

    n_batch = max([len(loader) for loader in aligned_train_loaders.values()])

    # Train the model
    all_weights = {
        "meta_train": {data_name: [] for data_name in meta_train_data_names},
        "meta_test": {data_name: [] for data_name in meta_test_data_names},
    }
    for i_epoch in range(n_epochs):
        prefix = f"Ep {i_epoch + 1}/{n_epochs}"
        # train over meta-train loaders
        train_losses = {data_name: [] for data_name in meta_train_data_names}
        # train_weights = {data_name: [] for data_name in meta_train_data_names}
        iterator = tqdm.tqdm(
            range(n_batch),
            total=n_batch,
            desc=f"{prefix} | meta-train ",
        )

        # turn loaders into iterators
        start_time = time.time()
        print("Turning loaders into iterators ... ", end="")
        meta_train_iterators = {
            data_name: iter(aligned_train_loaders[data_name])
            for data_name in meta_train_data_names
        }
        print(f"done in {time.time() - start_time:.2f}s")

        for i_batch in iterator:
            for train_name, meta_loader in meta_train_iterators.items():
                x_meta, y_model_preds, y_true = next(meta_loader)

                x_meta = (
                    scaler.transform(x_meta)  # Scale the meta-features
                    .float()
                    .to(device)
                )
                y_model_preds = y_model_preds.float().to(device)
                y_true = y_true.float().to(device)

                weights = fusor(x_meta)
                # train_weights[train_name].append(weights.detach().cpu().numpy())
                # Reshape weights to enable broadcasting with y_model_preds
                weights = weights.unsqueeze(-1).unsqueeze(-1)

                # Fuse the output by weighting model predictions
                # y_model_preds shape: (32, 14, 96, 7)
                # Resulting weighted shape: (32, 14, 96, 7)
                weighted_preds = weights * y_model_preds

                # Sum along the model dimension (dim=1) to get the fused output
                # fused_output shape: (32, 96, 7)
                fused_output = torch.sum(weighted_preds, dim=1)

                # Calculate loss and backpropagate
                loss = criterion(fused_output, y_true)
                train_loss = criterion(
                    fused_output[:, :training_step], y_true[:, :training_step]
                )
                optimizer.zero_grad()
                train_loss.backward()
                optimizer.step()

                # stop if weights contain NaN
                if torch.isnan(weights).any():
                    print("NaN weight detected")
                    raise ValueError

                train_losses[train_name].append(loss.item())

            if i_batch % 100 == 0:
                info = {
                    train_name: np.mean(train_losses[train_name])
                    for train_name in train_losses
                }
                iterator.set_postfix(**info)

        # update learning rate
        scheduler.step()

        # test over meta-test loaders

        meta_train_scores, meta_train_weights = test_fusor(
            fusor,
            scaler,
            meta_train_loaders,
            device,
        )
        meta_test_scores, meta_test_weights = test_fusor(
            fusor,
            scaler,
            meta_test_loaders,
            device,
        )
        for data_name in meta_train_loaders.keys():
            all_weights["meta_train"][data_name].append(meta_train_weights[data_name])
        for data_name in meta_test_loaders.keys():
            all_weights["meta_test"][data_name].append(meta_test_weights[data_name])
        print_test_scores(meta_test_scores, test_best_single_perf, ["mse", "mae"])


////// Forecast: [96, 48, 96] //////

Array has been loaded from './meta_data/ETTh1_val/x_meta_96.h5' (0.01s)
Array has been loaded from './meta_data/ETTh1_val/y_pred_96_48_96.h5' (0.35s)
Array has been loaded from './meta_data/ETTh1_val/y_true_96_48_96.h5' (0.04s)
Array has been loaded from './meta_data/ETTh2_val/x_meta_96.h5' (0.00s)
Array has been loaded from './meta_data/ETTh2_val/y_pred_96_48_96.h5' (0.32s)
Array has been loaded from './meta_data/ETTh2_val/y_true_96_48_96.h5' (0.04s)
Array has been loaded from './meta_data/ETTh1_test/x_meta_96.h5' (0.00s)
Array has been loaded from './meta_data/ETTh1_test/y_pred_96_48_96.h5' (0.32s)
Array has been loaded from './meta_data/ETTh1_test/y_true_96_48_96.h5' (0.03s)
Array has been loaded from './meta_data/ETTh2_test/x_meta_96.h5' (0.00s)
Array has been loaded from './meta_data/ETTh2_test/y_pred_96_48_96.h5' (0.33s)
Array has been loaded from './meta_data/ETTh2_test/y_true_96_48_96.h5' (0.03s)
Fitting the scaler for the meta-features ..

Ep 1/5 | meta-train :   0%|          | 0/44 [00:00<?, ?it/s]

Turning loaders into iterators ... 

Ep 1/5 | meta-train :   2%|▏         | 1/44 [00:00<00:07,  5.51it/s, ETTh1_val=0.92, ETTh2_val=0.333]

done in 0.10s


...

Turning loaders into iterators ... 

Ep 5/5 | meta-train :   2%|▏         | 1/44 [00:00<00:08,  5.37it/s, ETTh1_val=0.863, ETTh2_val=0.321]

done in 0.11s


Ep 5/5 | meta-train : 100%|██████████| 44/44 [00:00<00:00, 89.51it/s, ETTh1_val=0.863, ETTh2_val=0.321]
 meta-test  | ETTh2_val : 100%|██████████| 2/2 [00:00<00:00,  4.07it/s]
 meta-test  | ETTh2_test : 100%|██████████| 2/2 [00:00<00:00,  5.02it/s]

ETTh1_test | MSE: 0.3704 (-0.0079) | MAE: 0.3913 (-0.0068) | 
ETTh2_test | MSE: 0.2767 (-0.0087) | MAE: 0.3323 (-0.0053) | 


# [5] Results Print

Finally, we collect the results of TimeFuse and the base models, and print them in a table format.

We highlight the best model for each dataset and metric in green, the 2nd best in yellow, and the others in red.

In [ ]:
for data_name in model_scores.keys():  # add TimeFuse scores to model_scores
    model_scores[data_name]["TimeFuse (Ours)"] = meta_test_scores[data_name]

metrics = ["mse", "mae"]
print_models = ["TimeFuse (Ours)"] + run_configs["models"][::-1]
len_cell = 18

info = f"{'Dataset':<{len_cell}}{'Metric':<{len_cell}}"
for model_name in print_models:
    info += f"{model_name:<{len_cell}}"
print(info)

for data_name, data_scores in model_scores.items():
    for metric_name in metrics:
        info = f"{data_name:<{len_cell}}{metric_name.upper():<{len_cell}}"
        scores = [data_scores[model_name][metric_name] for model_name in print_models]
        model_score_ranks = np.argsort(scores)
        for i_model, model_name in enumerate(print_models):
            score = f"{data_scores[model_name][metric_name]:.4f}"
            if model_score_ranks[i_model] == 0:
                score = f"\033[1;32m{f'{score}**':<{len_cell}}\033[0m"  # Best score: green**
            elif model_score_ranks[i_model] == 1:
                score = f"\033[1;33m{f'{score}*':<{len_cell}}\033[0m"  # 2nd best score: yellow*
            else:
                score = f"\033[1;31m{score:<{len_cell}}\033[0m"  # 3rd best score: red
            info += f"{score:<{len_cell}}"
        print(info)

Dataset           Metric            TimeFuse (Ours)   TimeXer           TimeMixer         PAttn             TimesNet          PatchTST          DLinear           
ETTh1_test        MSE               0.3704**          0.3818            0.3784            0.3901*           0.3891            0.3793            0.3962            
ETTh1_test        MAE               0.3913**          0.4029            0.3981            0.4048*           0.4120            0.3996            0.4108            
ETTh2_test        MSE               0.2767**          0.2854*           0.2901            0.2988            0.3370            0.2921            0.3414            
ETTh2_test        MAE               0.3323**          0.3376*           0.3406            0.3453            0.3709            0.3452            0.3953            
